In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import re

In [2]:
import json

In [47]:
import ast
# LLM-based history update solution
import importlib.util
import inspect
from typing import List
import requests
import json
import re
import difflib
from collections import Counter
# from spellchecker import SpellChecker
from datetime import datetime
import pandas as pd
import ast
import random
import logging 
# from history_update_problem.call_or import export_rows
from call_or import *
from evaluation import *

# Worflow eval

In [396]:
def eval_workflows(pp_id, gt_wf_fp, pred_wf_fp):
    # print(gt_wf_fp)
    gt_ops_list = parse_recipe(pp_id, recipe=gt_wf_fp)
    pred_ops_list = parse_recipe(pp_id, recipe=pred_wf_fp)
    
    return {'pp_id': pp_id, 'gt_ops': gt_ops_list[pp_id], 'pred_ops': pred_ops_list[pp_id]}
    # print(gt_ops_list)
    # print(pred_ops_list)


models = ['llama3.1', 'gemma2', 'mistral']
answer_gt_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_small_table.json'
answer_preds_llama = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_llama3.1.json'


# eval_answer_results = eval_answers(answer_gt_path, answer_preds_llama)
model = models[1]
wf_gt_folder = '/projects/bces/lanl2/LLM4DC/datasets'
wf_pred_folder = f'/projects/bces/lanl2/LLM4DC/CoT.response/{model}/recipes_llm'

ops_list = []
query_contents = pd.read_csv('/projects/bces/lanl2/LLM4DC/purposes/queries.csv')
for query_id in range(111):
    row = query_contents[query_contents['ID'] == query_id]
    if len(row) == 0:
        continue
    if model == 'llama3.1':
        if query_id >= 62 and query_id <= 91:
            target_path = f'{wf_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/ppp_datasets/workflows/ppp_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/ppp_test_{query_id}.json"
        elif query_id >= 92:
            target_path = f'{wf_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/dish_datasets/workflows/dish_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/dish_test_{query_id}.json"
        elif query_id >= 31 and query_id <= 61:
            target_path = f'{wf_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/chi_food_inspection_datasets/workflows/chi_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/chi_test_{query_id}.json"
        elif query_id <31:
            target_path = f'{wf_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}'
            wf_gt_fp = f"{wf_gt_folder}/purpose-prepared-datasets/menu/workflows/menu_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/menu_test_{query_id}.json"
    else: 
        if query_id >= 62 and query_id <= 91:
            target_path = f'{wf_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/ppp_datasets/workflows/ppp_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_ppp_test_{query_id}.json"
        elif query_id >= 92:
            target_path = f'{wf_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/dish_datasets/workflows/dish_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_dish_test_{query_id}.json"
        elif query_id >= 31 and query_id <= 61:
            target_path = f'{wf_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
            wf_gt_fp = f"{wf_gt_folder}/chi_food_inspection_datasets/workflows/chi_sample_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_chi_test_{query_id}.json"
        elif query_id <31:
            target_path = f'{wf_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}'
            wf_gt_fp = f"{wf_gt_folder}/purpose-prepared-datasets/menu/workflows/menu_p{query_id}.json"
            wf_pred_fp = f"{wf_pred_folder}/{model}_menu_test_{query_id}.json"

    ops_list.append(eval_workflows(query_id, wf_gt_fp, wf_pred_fp))
ops_df = pd.DataFrame(ops_list)

In [397]:
ops_df['gt_ops_length'] = ops_df['gt_ops'].apply(len)
ops_df['pred_ops_length'] = ops_df['pred_ops'].apply(len)

In [398]:
ops_df['gt_ops_set_length'] = ops_df['gt_ops'].apply(lambda x: len(set(x)))
ops_df['pred_ops_set_length'] = ops_df['pred_ops'].apply(lambda x: len(set(x)))

In [399]:
ops_df.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,58.074627,5.268657,3.402985,2.925373,1.955224
std,35.047290,3.226698,2.322875,1.197389,0.944407
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,31.500000,3.000000,2.000000,2.000000,1.000000
50%,66.000000,5.000000,3.000000,3.000000,2.000000
75%,88.000000,6.000000,5.000000,4.000000,2.000000
max,110.000000,14.000000,10.000000,4.000000,4.000000


In [6]:
from evaluation import calculate_operation_metrics

In [400]:
workflow_results = calculate_operation_metrics(ops_df['gt_ops'], ops_df['pred_ops'])
workflow_results['pp_id'] = ops_df['pp_id']

In [386]:
workflow_results.describe()

,accuracy,precision,recall,f1,pp_id
count,67.000000,67.000000,67.000000,67.000000,67.000000
mean,0.104478,0.783582,0.611940,0.642360,58.074627
std,0.308188,0.284660,0.285211,0.228392,35.047290
min,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.666667,0.500000,0.571429,31.500000
50%,0.000000,1.000000,0.500000,0.666667,66.000000
75%,0.000000,1.000000,0.875000,0.775000,88.000000
max,1.000000,1.000000,1.000000,1.000000,110.000000


In [387]:
ppp_results = workflow_results[workflow_results['pp_id'] >= 62][workflow_results['pp_id'] <=91]
ppp_results.describe()

/tmp/ipykernel_671706/814825836.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = workflow_results[workflow_results['pp_id'] >= 62][workflow_results['pp_id'] <=91]


,accuracy,precision,recall,f1,pp_id
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.272727,0.799242,0.757576,0.734632,73.000000
std,0.455842,0.284822,0.285112,0.235405,7.438638
min,0.000000,0.000000,0.000000,0.000000,62.000000
25%,0.000000,0.666667,0.541667,0.666667,67.250000
50%,0.000000,1.000000,0.833333,0.666667,72.500000
75%,0.750000,1.000000,1.000000,0.964286,77.750000
max,1.000000,1.000000,1.000000,1.000000,89.000000


In [388]:
ppp_ops = ops_df[ops_df['pp_id'] >= 62][ops_df['pp_id'] <=91]
ppp_ops.describe()


/tmp/ipykernel_671706/3831109375.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_ops = ops_df[ops_df['pp_id'] >= 62][ops_df['pp_id'] <=91]


,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,73.000000,3.409091,3.863636,2.363636,2.136364
std,7.438638,1.842852,2.816541,1.255292,0.940894
min,62.000000,1.000000,1.000000,1.000000,1.000000
25%,67.250000,2.000000,2.000000,1.000000,1.250000
50%,72.500000,3.000000,3.000000,3.000000,2.000000
75%,77.750000,5.000000,4.000000,3.000000,3.000000
max,89.000000,6.000000,10.000000,4.000000,4.000000


In [389]:
dish_results = workflow_results[workflow_results['pp_id'] >= 92]
dish_results.describe()

,accuracy,precision,recall,f1,pp_id
count,16.0,16.000000,16.000000,16.000000,16.00000
mean,0.0,0.760417,0.515625,0.596429,101.93750
std,0.0,0.261539,0.213478,0.207858,5.65059
min,0.0,0.000000,0.000000,0.000000,92.00000
25%,0.0,0.666667,0.500000,0.571429,98.75000
50%,0.0,0.750000,0.500000,0.666667,102.50000
75%,0.0,1.000000,0.750000,0.750000,106.25000
max,0.0,1.000000,0.750000,0.857143,110.00000


In [391]:
dish_ops= ops_df[ops_df['pp_id'] >= 92]

dish_ops.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,16.00000,16.000000,16.000000,16.0000,16.000000
mean,101.93750,5.375000,5.625000,3.8125,2.687500
std,5.65059,1.310216,2.552776,0.7500,1.014479
min,92.00000,1.000000,1.000000,1.0000,1.000000
25%,98.75000,5.000000,3.750000,4.0000,2.000000
50%,102.50000,6.000000,6.000000,4.0000,3.000000
75%,106.25000,6.000000,8.000000,4.0000,3.250000
max,110.00000,7.000000,9.000000,4.0000,4.000000


In [401]:
chi_results = workflow_results[workflow_results['pp_id'] >= 31][workflow_results['pp_id'] <=61]
chi_results.describe()

/tmp/ipykernel_671706/2907689086.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_results = workflow_results[workflow_results['pp_id'] >= 31][workflow_results['pp_id'] <=61]


,accuracy,precision,recall,f1,pp_id
count,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.153846,0.692308,0.506410,0.538462,38.769231
std,0.375534,0.389956,0.347713,0.314126,6.260376
min,0.000000,0.000000,0.000000,0.000000,31.000000
25%,0.000000,0.500000,0.333333,0.400000,34.000000
50%,0.000000,1.000000,0.500000,0.500000,38.000000
75%,0.000000,1.000000,0.666667,0.666667,41.000000
max,1.000000,1.000000,1.000000,1.000000,52.000000


In [393]:
chi_ops = ops_df[ops_df['pp_id'] >= 31][ops_df['pp_id'] <=61]

chi_ops.describe()

/tmp/ipykernel_671706/2470253216.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_ops = ops_df[ops_df['pp_id'] >= 31][ops_df['pp_id'] <=61]


,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,13.000000,13.000000,13.000000,13.000000,13.000000
mean,38.769231,4.769231,5.153846,2.692308,2.230769
std,6.260376,2.047513,3.647971,1.031553,1.091928
min,31.000000,1.000000,1.000000,1.000000,1.000000
25%,34.000000,4.000000,3.000000,2.000000,1.000000
50%,38.000000,5.000000,4.000000,3.000000,2.000000
75%,41.000000,6.000000,9.000000,3.000000,3.000000
max,52.000000,8.000000,13.000000,4.000000,4.000000


In [394]:
menu_results = workflow_results[workflow_results['pp_id'] < 31]
menu_results.describe()

,accuracy,precision,recall,f1,pp_id
count,16.0000,16.000000,16.000000,16.000000,16.000000
mean,0.0625,0.802083,0.510417,0.579464,9.375000
std,0.2500,0.299498,0.280335,0.220044,5.667157
min,0.0000,0.000000,0.000000,0.000000,1.000000
25%,0.0000,0.625000,0.333333,0.500000,4.750000
50%,0.0000,1.000000,0.500000,0.666667,8.500000
75%,0.0000,1.000000,0.500000,0.666667,14.250000
max,1.0000,1.000000,1.000000,1.000000,18.000000


In [395]:
menu_ops = ops_df[ops_df['pp_id'] < 31]
menu_ops.describe()

,pp_id,gt_ops_length,pred_ops_length,gt_ops_set_length,pred_ops_set_length
count,16.000000,16.000000,16.000000,16.000000,16.0000
mean,9.375000,8.125000,5.625000,3.000000,1.6875
std,5.667157,4.688639,5.389805,1.154701,0.7932
min,1.000000,1.000000,0.000000,1.000000,0.0000
25%,4.750000,4.000000,1.750000,2.000000,1.0000
50%,8.500000,7.500000,3.500000,3.500000,2.0000
75%,14.250000,11.750000,8.500000,4.000000,2.0000
max,18.000000,14.000000,18.000000,4.000000,3.0000


# Dataset Eval

In [44]:
# def retrieve_tg_cols(tg_cols_fp="target_columns_list.csv"):
#     id_tg_cols = {}
#     tg_df = pd.read_csv(tg_cols_fp)
#     result_dict = tg_df.set_index('ID')['tg_columns'].to_dict()
#     return result_dict

In [340]:
result_dict = retrieve_tg_cols("/projects/bces/lanl2/LLM4DC/evaluation/target_columns_list.csv")
print(result_dict)
model = "llama3.1"
ratio_list = []
for query_id in range(111):
    # print(query_id)
    tg_cols = result_dict.get(query_id)
    if tg_cols:
        if model == "llama3.1":
            llm_folder = f"CoT.response/{model}/datasets_llm"
            data_gt_folder = "/projects/bces/lanl2/LLM4DC/datasets"
            pred_fp = f'/projects/bces/lanl2/LLM4DC/{llm_folder}'
            if query_id >= 62 and query_id <= 91:
                target_path = f'{data_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/ppp_test_{query_id}.csv'

            elif query_id >= 92:
                target_path = f'{data_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/dish_test_{query_id}.csv'

            elif query_id >= 31 and query_id <= 61:
                target_path = f'{data_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/chi_test_{query_id}.csv'

            elif query_id <31:
                target_path = f'{data_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/menu_test_{query_id}.csv'
        else:

            llm_folder = f"CoT.response/{model}/datasets_llm"
            data_gt_folder = "/projects/bces/lanl2/LLM4DC/datasets"
            pred_fp = f'/projects/bces/lanl2/LLM4DC/{llm_folder}'
            if query_id >= 62 and query_id <= 91:
                target_path = f'{data_gt_folder}/ppp_datasets/cleaned_tables/ppp_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_ppp_test_{query_id}.csv'

            elif query_id >= 92:
                target_path = f'{data_gt_folder}/dish_datasets/cleaned_tables/dish_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_dish_test_{query_id}.csv'

            elif query_id >= 31 and query_id <= 61:
                target_path = f'{data_gt_folder}/chi_food_inspection_datasets/cleaned_tables/chi_sample_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_chi_test_{query_id}.csv'

            elif query_id <31:
                target_path = f'{data_gt_folder}/purpose-prepared-datasets/menu/menu_p{query_id}.csv'
                table_preds_path = f'{pred_fp}/{model}_menu_test_{query_id}.csv'
        gt_df = pd.read_csv(target_path)
        preds_df = pd.read_csv(table_preds_path)
        # print(gt_df.head(5), preds_df.head(5))
        res = average_match_ratio(gt_df, preds_df, tg_cols)
        ratio_list.append({'pp_id': query_id, 'ratio': res})
dt_result = pd.DataFrame(ratio_list)

{1: 'page_count', 2: 'page_count', 3: 'event', 4: 'event', 5: 'event', 6: 'venue', 7: 'occasion', 8: 'occasion', 9: 'page_count, dish_count', 12: 'page_count, location', 13: 'sponsor, currency', 14: 'sponsor, dish_count', 15: 'sponsor, event', 16: 'sponsor, event', 17: 'sponsor, event', 18: 'sponsor, event', 31: 'Risk', 32: 'Results', 33: 'Facility Type', 34: 'Facility Type', 36: 'DBA Name, Results', 37: 'DBA Name, Results', 38: 'Facility Type, Risk', 39: 'Facility Type, Risk', 40: 'Facility Type, Risk', 41: 'Facility Type, Risk', 42: 'Facility Type, Risk', 49: 'Facility Type, Risk, Results', 52: 'Address, Risk, Results', 62: 'LoanAmount', 63: 'LoanAmount', 64: 'LoanAmount', 65: 'NAICSCode, JobsReported', 66: 'JobsReported, LoanAmount', 67: 'City', 68: 'BusinessType, LoanAmount', 69: 'BusinessType, LoanAmount', 70: 'BusinessType, LoanAmount', 71: 'BusinessType, Zip', 72: 'Zip, LoanAmount', 73: 'Zip, LoanAmount', 74: 'Gender, LoanAmount', 75: 'Gender, LoanAmount', 76: 'City, LoanAmount'

In [341]:
dt_result.head()

,pp_id,ratio
0,1,0.30
1,2,1.00
2,3,0.25
3,4,0.37
4,5,0.37


In [342]:
dt_result.to_csv(f'evaluation/{model}_table_result.csv')

In [343]:
dt_result.head()

,pp_id,ratio
0,1,0.30
1,2,1.00
2,3,0.25
3,4,0.37
4,5,0.37


In [344]:
stat = dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.7850146992311171
0.2415813137609917
0.17
1.0
0.6266666666666667
0.875
0.98125


In [345]:
ppp_dt_results = dt_result[dt_result['pp_id'] >= 62][dt_result['pp_id'] <=91]
stat = ppp_dt_results['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.8861742424242425
0.16056997087980227
0.5
1.0
0.85625
0.95
1.0


/tmp/ipykernel_671706/1140604443.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_dt_results = dt_result[dt_result['pp_id'] >= 62][dt_result['pp_id'] <=91]


In [346]:
dish_dt_result =dt_result[dt_result['pp_id'] >= 92]
stat = dish_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.8260416666666667
0.18342914163240256
0.2866666666666667
1.0
0.87
0.8766666666666667
0.9233333333333333


In [347]:
chi_dt_result =dt_result[dt_result['pp_id'] >= 31][dt_result['pp_id'] <=61]
stat = chi_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.8460372960372958
0.1732111979435169
0.45
1.0
0.7666666666666666
0.85
1.0


/tmp/ipykernel_671706/3331208062.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_dt_result =dt_result[dt_result['pp_id'] >= 31][dt_result['pp_id'] <=61]


In [348]:
menu_dt_result =dt_result[dt_result['pp_id'] < 31]

stat = menu_dt_result['ratio'].describe()
for idx in ['mean', 'std',	'min',	'max',	'25%',	'50%',	'75%']:
    print(stat.loc[idx])

0.5553124999999999
0.2945078875570794
0.17
1.0
0.3075
0.54
0.7525


## table eval ttest

# answer eval

In [206]:
datafile_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_gt.json'
data = []
with open(datafile_path, 'r') as f:
    for l in f:
        data.append(json.loads(l))

In [207]:
from bert_score import score

In [220]:
import json
from typing import Union, List, Dict, Any
from difflib import SequenceMatcher
from math import isclose

# Define a utility to convert JSON strings to Python objects
def parse_input(answer: Union[str, float, List, Dict]) -> Any:
    if isinstance(answer, str):
        try:
            # Try to parse JSON strings into Python objects
            return json.loads(answer)
        except json.JSONDecodeError:
            return answer.lower().strip()  # Normalize strings for comparison
    elif isinstance(answer, float):
        return round(answer, 2)  # Round floats to two decimal places if needed
    return answer  # If already in desired format

# Calculate exact match accuracy
def accuracy_metric(gt: Any, pred: Any) -> float:
    return 1.0 if gt == pred else 0.0

# Calculate precision, recall, and F1 for lists (assuming items are unique)
def precision_recall_f1(gt: List, pred: List) -> Dict[str, float]:
    gt_set, pred_set = set(gt), set(pred)
    true_positives = len(gt_set & pred_set)
    precision = true_positives / len(pred_set) if pred_set else 0
    recall = true_positives / len(gt_set) if gt_set else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return {"precision": precision, "recall": recall, "f1": f1_score}

# Calculate semantic distance for string answers using sequence matching
def semantic_similarity(gt: str, pred: str) -> float:
    return SequenceMatcher(None, gt, pred).ratio()  # Returns a ratio between 0 and 1

# Evaluate an answer based on the ground truth
def calculate_answer_metrics(gt: Any, pred: Any) -> Dict[str, float]:
    # Parse inputs
    gt, pred = parse_input(gt), parse_input(pred)
    
    # Initialize results
    results = {"accuracy": 0, "semantic_similarity": 0, "precision": 0, "recall":0, "f1":0}
    
    # Check type and apply appropriate metrics
    if isinstance(gt, float) and isinstance(pred, float):
        results["accuracy"] = 1.0 if isclose(gt, pred, rel_tol=1e-2) else 0.0  # Accuracy for floats with tolerance
        precision, recall = results["accuracy"], results["accuracy"]
        results.update({"precision": precision, "recall": recall, "f1": 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0})
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
        results["semantic_similarity"] = semantic_similarity(str(gt), str(pred))


    elif isinstance(gt, str) and isinstance(pred, str):
        results["accuracy"] = accuracy_metric(gt, pred)
        precision, recall = results["accuracy"], results["accuracy"]
        results.update({"precision": precision, "recall": recall, "f1": 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0})
        # results['bertscore'] = score([pred], [gt], lang='en', verbose=True)
        results["semantic_similarity"] = semantic_similarity(gt, pred)

    
    elif isinstance(gt, list) and isinstance(pred, list):
        metrics = precision_recall_f1(gt, pred)
        results.update(metrics)
        results["accuracy"] = accuracy_metric(gt, pred)
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
        similarity = semantic_similarity(f"{gt}", f"{pred}")
        results["semantic_similarity"] = similarity

    
    elif isinstance(gt, dict) and isinstance(pred, dict):
        gt_keys, pred_keys = list(gt.keys()), list(pred.keys())
        results.update(precision_recall_f1(gt_keys, pred_keys))  # Precision/Recall on keys
        results["accuracy"] = accuracy_metric(gt, pred)
        # Check semantic similarity for each key-value pair
        similarity = [semantic_similarity(str(gt[k]), str(pred.get(k, ""))) for k in gt_keys]
        results["semantic_similarity"] = sum(similarity) / len(similarity) if similarity else 0
        
        # results['bertscore'] = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
    p, r, f1 = score([f"{pred}"], [f"{gt}"], lang='en', verbose=True)
    results.update({'bertscore_p': p.detach().cpu().tolist(),
    'bertscore_r': r.detach().cpu().tolist(),
    'bertscore_f1': f1.detach().cpu().tolist()})
    # print(results)


    return results

In [86]:
test_json = "{\"Zip\":{\"0\":96701,\"1\":96704,\"2\":96707,\"3\":96708,\"4\":96749,\"5\":96750,\"6\":96754,\"7\":96791,\"8\":96813,\"9\":96814,\"10\":96815,\"11\":96816,\"12\":96817,\"13\":96821,\"14\":96825,\"15\":96826},\"LoanCount\":{\"0\":1,\"1\":1,\"2\":4,\"3\":1,\"4\":1,\"5\":1,\"6\":1,\"7\":1,\"8\":1,\"9\":1,\"10\":1,\"11\":2,\"12\":1,\"13\":1,\"14\":1,\"15\":1}}"

In [221]:
# LLM-based history update solution
import importlib.util
import inspect
from typing import List
import requests
import json
import re
import difflib
from collections import Counter
# from spellchecker import SpellChecker
from datetime import datetime
import pandas as pd
import ast
import random
import logging 
# from history_update_problem.call_or import export_rows
from call_or import *

# from evaluation.data_compare import calculate_answer_metrics


def load_answer_dataset(datafile_path):
    """
    load json file, each line is a json dictionary

    datafile_path: str
    return: data:  list_of_dictionary
    """
    data = []
    with open(datafile_path, 'r') as f:
        for l in f:
            data.append(json.loads(l))
    return data

def eval_answers(answer_gt_path, answer_preds_llama):
    answer_gt = load_answer_dataset(answer_gt_path)
    answer_gt = pd.DataFrame(answer_gt)
    answer_preds_llama = load_answer_dataset(answer_preds_llama)
    answer_preds_llama = pd.DataFrame(answer_preds_llama)
    answer_compare = answer_gt.merge(answer_preds_llama[['pp_id', 'answer']], on='pp_id', how='left', suffixes=('_gt', '_preds'))

    results = []
    for i, row in answer_compare.iterrows():
        gt = row['answer_gt']
        preds = row['answer_preds']
        single_result = calculate_answer_metrics(gt, preds)
        single_result['pp_id'] = row['pp_id']
        results.append(single_result)
        # break
    return pd.DataFrame(results)

 
    

In [402]:
answer_gt_path = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_gt.json'
answer_preds_llama = '/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_llama3.1.json'
model = 'gemma2'
answer_preds = f'/projects/bces/lanl2/LLM4DC/evaluation/answer_1-110_{model}.json'

eval_answer_results = eval_answers(answer_gt_path, answer_preds)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 98.55it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 873.45it/s]

...

calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 35.06it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 724.40it/s]

done in 0.04 seconds, 26.80 sentences/sec


In [266]:
eval_answer_results.tail()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
62,0.0,0.965517,0.0,0.0,0.0,[0.921100378036499],[0.9549583196640015],[0.9377238154411316],106
63,0.0,0.000000,0.0,0.0,0.0,[0.0],[0.0],[0.0],107
64,1.0,1.000000,1.0,1.0,1.0,[0.9999998211860657],[0.9999998211860657],[0.9999998211860657],108
65,0.0,0.995951,0.8,0.8,0.8,[0.9935122132301331],[0.9978277683258057],[0.9956653118133545],109
66,0.0,0.792194,1.0,1.0,1.0,[0.9665622711181641],[0.9755587577819824],[0.9710396528244019],110


In [267]:
eval_answer_results['bertscore_p'] = eval_answer_results['bertscore_p'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)
eval_answer_results['bertscore_r'] = eval_answer_results['bertscore_r'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)
eval_answer_results['bertscore_f1'] = eval_answer_results['bertscore_f1'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else None)


In [268]:
eval_answer_results.to_csv(f'/projects/bces/lanl2/LLM4DC/evaluation/{model}_answer_result.csv')

In [408]:
model = 'mistral'

In [409]:
eval_answer_results = pd.read_csv(f'/projects/bces/lanl2/LLM4DC/evaluation/{model}_answer_result.csv')

In [410]:
eval_answer_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,33.000000,0.223881,0.481756,0.406553,0.391041,0.395572,0.952298,0.944111,0.947865,58.074627
std,19.485037,0.419989,0.399946,0.482701,0.472106,0.473316,0.059367,0.073427,0.065150,35.047290
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.780360,0.734762,0.758963,1.000000
25%,16.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.914918,0.905095,0.917902,31.500000
50%,33.000000,0.000000,0.461538,0.000000,0.000000,0.000000,0.980539,0.981595,0.980539,66.000000
75%,49.500000,0.000000,0.936486,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,88.000000
max,66.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,110.000000


In [271]:
ppp_results = eval_answer_results[eval_answer_results['pp_id'] >= 62][eval_answer_results['pp_id'] <=91]

/tmp/ipykernel_671706/2424205661.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ppp_results = eval_answer_results[eval_answer_results['pp_id'] >= 62][eval_answer_results['pp_id'] <=91]


In [273]:
dish_results = eval_answer_results[eval_answer_results['pp_id'] >= 92]

In [411]:
chi_results = eval_answer_results[eval_answer_results['pp_id'] >= 31][eval_answer_results['pp_id'] <=61]

/tmp/ipykernel_671706/3367146683.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chi_results = eval_answer_results[eval_answer_results['pp_id'] >= 31][eval_answer_results['pp_id'] <=61]


In [275]:
menu_results = eval_answer_results[eval_answer_results['pp_id'] < 31]

In [276]:
ppp_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,0.500000,0.737315,0.623377,0.623377,0.623377,0.976612,0.977359,0.976741,73.000000
std,0.511766,0.369270,0.486045,0.486045,0.486045,0.047354,0.038682,0.041231,7.438638
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.800825,0.887636,0.857735,62.000000
25%,0.000000,0.534770,0.000000,0.000000,0.000000,0.979969,0.979969,0.979969,67.250000
50%,0.500000,0.991803,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,72.500000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,77.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,89.000000


In [277]:
dish_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000
mean,0.312500,0.752926,0.657052,0.643437,0.649228,0.894152,0.890705,0.892202,101.93750
std,0.478714,0.345467,0.461888,0.459408,0.459677,0.244095,0.248477,0.245975,5.65059
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,92.00000
25%,0.000000,0.498152,0.000000,0.000000,0.000000,0.914238,0.923151,0.921690,98.75000
50%,0.000000,0.969938,0.937500,0.937500,0.937500,0.965621,0.972786,0.968748,102.50000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,106.25000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,110.00000


In [278]:
menu_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.187500,0.368430,0.345170,0.409231,0.349199,0.968669,0.978194,0.973050,9.375000
std,0.403113,0.453404,0.458242,0.489499,0.440194,0.060247,0.039421,0.048239,5.667157
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.815612,0.885391,0.853846,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.975941,0.976899,0.971266,4.750000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.999803,0.999803,0.999803,8.500000
75%,0.000000,0.869799,0.954545,1.000000,0.788462,1.000000,1.000000,1.000000,14.250000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18.000000


In [412]:
chi_results.describe()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
count,13.00000,13.0,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,22.00000,0.0,0.353690,0.282051,0.242308,0.252150,0.962000,0.942052,0.951449,38.769231
std,3.89444,0.0,0.362727,0.444277,0.404066,0.405938,0.053973,0.082457,0.067272,6.260376
min,16.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.816275,0.734762,0.773377,31.000000
25%,19.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.953300,0.907506,0.930171,34.000000
50%,22.00000,0.0,0.210981,0.000000,0.000000,0.000000,0.984460,0.977580,0.981008,38.000000
75%,25.00000,0.0,0.666667,0.750000,0.400000,0.571429,1.000000,1.000000,1.000000,41.000000
max,28.00000,0.0,0.918919,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,52.000000


In [43]:
eval_answer_results = pd.DataFrame(eval_answer_results)

In [40]:
eval_answer_results['accuracy'].sum()

np.float64(21.0)

In [100]:
eval_answer_results.describe()

,accuracy,semantic_similarity,precision,recall,f1,pp_id
count,67.000000,67.0,67.000000,67.000000,67.000000,67.000000
mean,0.298507,0.0,0.444066,0.442419,0.442747,58.074627
std,0.461057,0.0,0.485182,0.484086,0.483975,35.047290
min,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,31.500000
50%,0.000000,0.0,0.000000,0.000000,0.000000,66.000000
75%,1.000000,0.0,1.000000,1.000000,1.000000,88.000000
max,1.000000,0.0,1.000000,1.000000,1.000000,110.000000


## answer eval ttest


In [281]:
from scipy import stats

In [282]:
gemma2_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/gemma2_answer_result.csv')
llama_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/llama3.1_answer_result.csv')
mistral_answer_result = pd.read_csv('/projects/bces/lanl2/LLM4DC/evaluation/mistral_answer_result.csv')

In [283]:
llama_answer_result.head()

,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1,pp_id
0,0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,1
1,1,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,2
2,2,0.0,0.0,0.0,0.0,0.0,0.999803,0.999803,0.999803,3
3,3,0.0,0.0,0.0,0.0,0.0,0.999268,0.999268,0.999268,4
4,4,0.0,0.0,0.0,0.0,0.0,0.999813,0.999813,0.999813,5


In [288]:
answer_result = llama_answer_result.merge(gemma2_answer_result, on='pp_id', how='left', suffixes=('_llama', '_gemma2')).merge(mistral_answer_result, on='pp_id', how='left', suffixes=('', '_mistral'))

In [295]:
metric_names = ['accuracy','semantic_similarity', 'precision', 'recall', 'f1', 'bertscore_p', 'bertscore_r','bertscore_f1']


In [309]:
ttest_result = []
for m in metric_names:
    model_list = ['llama', 'gemma2', 'mistral']
    gemma_result = stats.ttest_rel(answer_result[f'{m}_llama'].values, answer_result[f'{m}_gemma2'].values)
    # print(answer_result[f'{m}_llama'].values)
    mistral_result = stats.ttest_ind(answer_result[f'{m}_llama'].values, answer_result[f'{m}'].values)
    ttest_result.append({f'{m}':[gemma_result.pvalue.item(), mistral_result.pvalue.item()]})
    # print(gemma_result)
    # print(mistral_result)
    # break

In [311]:
combined_data = {k: v for d in ttest_result for k, v in d.items()}

ttest_result = pd.DataFrame(combined_data)

In [314]:
ttest_result

,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,1.000000,0.113517,0.270473,0.210301,0.291698,0.541825,0.926403,0.793618
1,0.329154,0.213945,0.271178,0.170657,0.234213,0.898096,0.890513,0.993298


In [289]:
answer_result

,Unnamed: 0_llama,accuracy_llama,semantic_similarity_llama,precision_llama,recall_llama,f1_llama,bertscore_p_llama,bertscore_r_llama,bertscore_f1_llama,pp_id,...,bertscore_f1_gemma2,Unnamed: 0,accuracy,semantic_similarity,precision,recall,f1,bertscore_p,bertscore_r,bertscore_f1
0,0,0.0,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,1,...,1.000000,0,0.0,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000
1,1,0.0,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,2,...,1.000000,1,0.0,0.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000
2,2,0.0,0.000000,0.0,0.0,0.0,0.999803,0.999803,0.999803,3,...,0.999477,2,0.0,0.000000,0.0,0.0,0.0,0.999477,0.999477,0.999477
3,3,0.0,0.000000,0.0,0.0,0.0,0.999268,0.999268,0.999268,4,...,0.999268,3,0.0,0.000000,0.0,0.0,0.0,0.999349,0.999349,0.999349
4,4,0.0,0.000000,0.0,0.0,0.0,0.999813,0.999813,0.999813,5,...,0.978343,4,0.0,0.000000,0.0,0.0,0.0,0.999813,0.999813,0.999813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,0.0,0.965517,0.0,0.0,0.0,0.921100,0.954958,0.937724,106,...,1.000000,62,0.0,0.808511,0.0,0.0,0.0,0.951192,0.894924,0.922200
63,63,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,107,...,0.822745,63,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
64,64,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,108,...,0.942593,64,0.0,0.000000,0.0,0.0,0.0,0.874944,0.780197,0.824859
65,65,0.0,0.995951,0.8,0.8,0.8,0.993512,0.997828,0.995665,109,...,0.876408,65,0.0,0.356275,0.0,0.0,0.0,0.904751,0.872501,0.888333


In [4]:
def json_to_custom_string(data):
    # Serialize JSON with indentation for readability within each nested structure
    json_str = json.dumps(data, indent=None, separators=(r'')
    
    # Insert newline only after each top-level key-value pair
    # This keeps inner lists and dicts intact without adding extra line breaks
    formatted_str = json_str[1:-1].replace('", "', '",\n"')  # Exclude outer braces for custom formatting
    return f"{{\n{formatted_str}\n}}"


In [ ]:
json.dumps(data, indent=1, sepera)

In [11]:
data = {'table_caption': 'A mix of simple bibliographic description of the menus', 'columns': ['id', 'name', 'sponsor', 'event', 'venue', 'place', 'physical_description', 'occasion', 'notes', 'call_number', 'keywords', 'language', 'date', 'location', 'location_type', 'currency', 'currency_symbol', 'status', 'page_count', 'dish_count'], 'table_column_priority': [['id', '12579', '25121', '21960'], ['name', '', '', ''], ['sponsor', 'TRUSTEES OF THE MISSOURI BOTANICAL GARDEN', 'HOLLAND HOUSE', 'BATTERY PARK HOTEL'], ['event', '11TH ANNUAL BANQUET', 'LUNCHEON', 'CHRISTMAS DINNER'], ['venue', 'PROF;', 'COMMERCIAL', 'COMMERCIAL'], ['place', 'SOUTHERN HOTEL,ST. LOUIS,MO.', '', 'ASHVILLE, NC'], ['physical_description', 'BROADSIDE; ILLUS; 5.5 X 8.75;', 'CARD;6X8.75;', 'BOOKLET; ILLUS; COL; 6 X 8;'], ['occasion', 'ANNUAL', 'DAILY;', 'RELIGIOUS HOLIDAY'], ['notes', 'WINES LISTED FOR EACH COURSE;', 'ENGLISH ON ONE SIDE,FRENCH ON ONE SIDE;', 'PRINTED ON PARCHMENT-LIKE PAPER; RED LION; DRAWING OF HOTEL; TIED WITH TWINE AND SEALING WAX; MOST OF BOOKLET CONSISTS OF INFORMATION AND PRAISE ABOUT THE HOTEL;'], ['call_number', '1900-2627', '1900-517', '1898-432'], ['keywords', '', '', ''], ['language', '', '', ''], ['date', '1900-03-31', '1900-01-25', '1898-12-25'], ['location', 'Trustees Of The Missouri Botanical Garden', 'Holland House', 'Battery Park Hotel'], ['location_type', '', '', ''], ['currency', '', 'Dollars', ''], ['currency_symbol', '', '$', ''], ['status', 'complete', 'complete', 'complete'], ['page_count', '2', '2', '16'], ['dish_count', '22.0', '546.0', '28.0']]}


In [27]:
data_str = '\{\n'
for key, value in data.items():
    if key in ['table_caption', 'columns']:
        data_str += f'{key}: {value},\n'
    else:
        data_str += key + ': [\n'
        for l in value:
            data_str += f'  {l},\n'
data_str += '  ]\n}'
    


In [31]:
type(data_str)

str

In [29]:
pprint.pprint(data_str)

('{\n'
 'table_caption: A mix of simple bibliographic description of the menus,\n'
 "columns: ['id', 'name', 'sponsor', 'event', 'venue', 'place', "
 "'physical_description', 'occasion', 'notes', 'call_number', 'keywords', "
 "'language', 'date', 'location', 'location_type', 'currency', "
 "'currency_symbol', 'status', 'page_count', 'dish_count'],\n"
 'table_column_priority: [\n'
 "  ['id', '12579', '25121', '21960'],\n"
 "  ['name', '', '', ''],\n"
 "  ['sponsor', 'TRUSTEES OF THE MISSOURI BOTANICAL GARDEN', 'HOLLAND HOUSE', "
 "'BATTERY PARK HOTEL'],\n"
 "  ['event', '11TH ANNUAL BANQUET', 'LUNCHEON', 'CHRISTMAS DINNER'],\n"
 "  ['venue', 'PROF;', 'COMMERCIAL', 'COMMERCIAL'],\n"
 "  ['place', 'SOUTHERN HOTEL,ST. LOUIS,MO.', '', 'ASHVILLE, NC'],\n"
 "  ['physical_description', 'BROADSIDE; ILLUS; 5.5 X 8.75;', 'CARD;6X8.75;', "
 "'BOOKLET; ILLUS; COL; 6 X 8;'],\n"
 "  ['occasion', 'ANNUAL', 'DAILY;', 'RELIGIOUS HOLIDAY'],\n"
 "  ['notes', 'WINES LISTED FOR EACH COURSE;', 'ENGLISH ON ON

In [10]:
import pandas as pd

In [7]:
import pprint

In [8]:
pprint.pprint(json_to_custom_string(data))

('{\n'
 '"table_caption": "A mix of simple bibliographic description of the '
 'menus","columns": '
 '["id","name","sponsor","event","venue","place","physical_description","occasion","notes","call_number","keywords","language","date","location","location_type","currency","currency_symbol","status","page_count","dish_count"],"table_column_priority": '
 '[["id","12579","25121","21960"],["name","","",""],["sponsor","TRUSTEES OF '
 'THE MISSOURI BOTANICAL GARDEN","HOLLAND HOUSE","BATTERY PARK '
 'HOTEL"],["event","11TH ANNUAL BANQUET","LUNCHEON","CHRISTMAS '
 'DINNER"],["venue","PROF;","COMMERCIAL","COMMERCIAL"],["place","SOUTHERN '
 'HOTEL,ST. LOUIS,MO.","","ASHVILLE, NC"],["physical_description","BROADSIDE; '
 'ILLUS; 5.5 X 8.75;","CARD;6X8.75;","BOOKLET; ILLUS; COL; 6 X '
 '8;"],["occasion","ANNUAL","DAILY;","RELIGIOUS HOLIDAY"],["notes","WINES '
 'LISTED FOR EACH COURSE;","ENGLISH ON ONE SIDE,FRENCH ON ONE SIDE;","PRINTED '
 'ON PARCHMENT-LIKE PAPER; RED LION; DRAWING OF HOTEL; TIED WI

In [28]:
paragraph = "[    {'from': ['Luncheon', 'lunch'], 'to': 'Luncheon'},    {'from': ['Dinner'], 'to': 'Dinner'},  # No change needed for Dinner    {'from': ['Breakfast'], 'to': 'Breakfast'},  # No change needed for Breakfast    {'from': ['', 'Unknown', None], 'to': 'Unknown'},  # Replace missing values with Unknown    {'from': ['Banquet', 'Annual Banquet'], 'to': 'Annual Banquet'},    {'from': ['Tiffin'], 'to': 'Tiffin'}  # No change needed for Tiffin]"

In [3]:
raw_string = """```python
[
    {'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},
    {'from': ['KANEOHE'], 'to': 'Kaneohe'},  # corrected spelling
    {'from': ['KIHEI'], 'to': 'Kihei'},  # corrected spelling
    {'from': ['Kailua Kona', 'KAUNAKAKAI', 'WAIMANALO', 'Waimea', 'kalaheo', 'Kahului'], 'to': 'Unknown'},  # unknown cities
    {'from': ['HONOLULUulu'], 'to': 'Honolulu'}  # corrected spelling
]
```"""

In [4]:
result = re.findall(r'(\[(:?\n?.*\n?)*\])', raw_string, re.DOTALL)

In [5]:
if result:
        for r in result:
            raw_string = r[0]

In [6]:
raw_string

"[\n    {'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},\n    {'from': ['KANEOHE'], 'to': 'Kaneohe'},  # corrected spelling\n    {'from': ['KIHEI'], 'to': 'Kihei'},  # corrected spelling\n    {'from': ['Kailua Kona', 'KAUNAKAKAI', 'WAIMANALO', 'Waimea', 'kalaheo', 'Kahului'], 'to': 'Unknown'},  # unknown cities\n    {'from': ['HONOLULUulu'], 'to': 'Honolulu'}  # corrected spelling\n]"

In [7]:
eval(raw_string)

[{'from': ['HONOLULU', 'Honolulu'], 'to': 'Honolulu'},
 {'from': ['KANEOHE'], 'to': 'Kaneohe'},
 {'from': ['KIHEI'], 'to': 'Kihei'},
 {'from': ['Kailua Kona',
   'KAUNAKAKAI',
   'WAIMANALO',
   'Waimea',
   'kalaheo',
   'Kahului'],
  'to': 'Unknown'},
 {'from': ['HONOLULUulu'], 'to': 'Honolulu'}]

In [26]:
paragraph = re.sub(r'#.*\{$', '{', paragraph, flags=re.MULTILINE)


In [29]:
result = re.findall(r'(\[(:?\n?(^#.*\{|]}).*\n?)*\])', paragraph, re.DOTALL)


In [30]:
line = "{'city':['hhhh'] #????ddfgtecg"

[]

In [5]:
input = '''
[{'foo':'bar'} # 000asd
]'''
eval(input)


[{'foo': 'bar'}]

In [27]:
paragraph

"[    {'from': ['Luncheon', 'lunch'], 'to': 'Luncheon'},    {'from': ['Dinner'], 'to': 'Dinner'},  # No change needed for Dinner    {'from': ['Breakfast'], 'to': 'Breakfast'},  # No change needed for Breakfast    {'from': ['', 'Unknown', None], 'to': 'Unknown'},  # Replace missing values with Unknown    {'from': ['Banquet', 'Annual Banquet'], 'to': 'Annual Banquet'},    {'from': ['Tiffin'], 'to': 'Tiffin'}  # No change needed for Tiffin]"

In [1]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Extract column headers and index to form the first row in the table
    columns = ["competition"] + df.columns.tolist()
    
    # Prepare rows by combining index and corresponding row values in the DataFrame
    table_column_priority = [columns]  # Start with header row
    for idx, row in df.iterrows():
        table_column_priority.append([idx] + row.tolist())
    
    # Create the dictionary in the required format
    table_data = {
        "table_caption": table_caption,
        "columns": columns,
        "table_column_priority": table_column_priority
    }
    
    # Convert to JSON string with indentation for readability
    return json.dumps(table_data, indent=2)

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format and print
table_format_json = df_to_table_format(df, "south wales derby")
print(table_format_json)


{
  "table_caption": "south wales derby",
  "columns": [
    "competition",
    "total matches",
    "cardiff win",
    "draw",
    "swansea win"
  ],
  "table_column_priority": [
    [
      "competition",
      "total matches",
      "cardiff win",
      "draw",
      "swansea win"
    ],
    [
      "league",
      "55",
      "19",
      "16",
      "20"
    ],
    [
      "fa cup",
      "2",
      "0",
      "27",
      "2"
    ],
    [
      "league cup",
      "5",
      "2",
      "0",
      "3"
    ]
  ]
}


In [3]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Extract column headers and index to form the first row in the table
    columns = ["competition"] + df.columns.tolist()
    
    # Prepare rows by combining index and corresponding row values in the DataFrame
    table_column_priority = [columns]  # Start with header row
    for idx, row in df.iterrows():
        table_column_priority.append([idx] + row.tolist())
    
    # Create the dictionary in the required format
    table_data = {
        "table_caption": table_caption,
        "columns": columns,
        "table_column_priority": table_column_priority
    }
    
    # Format 'columns' as a compact list and use json.dumps for the rest
    columns_str = f'"columns": {json.dumps(columns)}'
    table_column_priority_str = f'"table_column_priority": {json.dumps(table_column_priority)}'
    caption_str = f'"table_caption": "{table_caption}"'
    
    # Concatenate each component into a final JSON format
    final_output = f'{{\n  {caption_str},\n  {columns_str},\n  {table_column_priority_str}\n}}'
    
    return final_output

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format and print
table_format_json = df_to_table_format(df, "south wales derby")
print(table_format_json)


{
  "table_caption": "south wales derby",
  "columns": ["competition", "total matches", "cardiff win", "draw", "swansea win"],
  "table_column_priority": [["competition", "total matches", "cardiff win", "draw", "swansea win"], ["league", "55", "19", "16", "20"], ["fa cup", "2", "0", "27", "2"], ["league cup", "5", "2", "0", "3"]]
}


In [4]:
import pandas as pd
import json

def df_to_table_format(df, table_caption):
    # Prepare rows for table_column_priority
    table_column_priority = []
    
    # The first list is the competition names (the DataFrame index)
    table_column_priority.append(["competition"] + df.index.tolist())
    
    # Append each DataFrame column values to table_column_priority
    for col in df.columns:
        table_column_priority.append([col] + df[col].tolist())
    
    # Create the final dictionary
    table_data = {
        "table_caption": table_caption,
        "columns": ["competition"] + df.columns.tolist(),
        "table_column_priority": table_column_priority
    }
    
    return table_data

# Example DataFrame
data = {
    "total matches": ["55", "2", "5"],
    "cardiff win": ["19", "0", "2"],
    "draw": ["16", "27", "0"],
    "swansea win": ["20", "2", "3"]
}
index = ["league", "fa cup", "league cup"]

df = pd.DataFrame(data, index=index)

# Convert DataFrame to the specified format
table_format = df_to_table_format(df, "south wales derby")

# Print the output in the desired format
print(json.dumps(table_format, indent=2))


{
  "table_caption": "south wales derby",
  "columns": [
    "competition",
    "total matches",
    "cardiff win",
    "draw",
    "swansea win"
  ],
  "table_column_priority": [
    [
      "competition",
      "league",
      "fa cup",
      "league cup"
    ],
    [
      "total matches",
      "55",
      "2",
      "5"
    ],
    [
      "cardiff win",
      "19",
      "0",
      "2"
    ],
    [
      "draw",
      "16",
      "27",
      "0"
    ],
    [
      "swansea win",
      "20",
      "2",
      "3"
    ]
  ]
}
